<a href="https://colab.research.google.com/github/ProgramasParaFisicaDeParticulas/lecturadedatosabiertoscmszmumu-MPM-cvr/blob/main/CMSOpenDataZtoMuMu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
#panda nos permite manipular el archivo csv https://pandas.pydata.org
import pandas as pd
#numpy nos permite hacer operaciones usando vectores https://numpy.org
import numpy as np
#plotly nos permite crear graficas interactivas https://plotly.com/python/
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from scipy.stats import cauchy
from scipy.optimize import curve_fit
from scipy.signal import fftconvolve

%matplotlib inline
#usamos la liga que está directamente en la página.
particles = pd.read_csv('https://opendata.cern.ch/record/5208/files/Zmumu.csv',delimiter=',')
particles



,Run,Event,pt1,eta1,phi1,Q1,dxy1,iso1,pt2,eta2,phi2,Q2,dxy2,iso2
0,165617,74969122,54.7055,-0.4324,2.5742,1,-0.0745,0.4999,34.2464,-0.9885,-0.4987,-1,0.0712,3.4221
1,165617,75138253,24.5872,-2.0522,2.8666,-1,-0.0554,0.0000,28.5389,0.3852,-1.9912,1,0.0515,0.0000
2,165617,75887636,31.7386,-2.2595,-1.3323,-1,0.0879,0.0000,30.2344,-0.4684,1.8833,1,-0.0876,0.0000
3,165617,75779415,39.7394,-0.7123,-0.3123,1,0.0585,0.0000,48.2790,-0.1956,2.9703,-1,-0.0492,0.0000
4,165617,75098104,41.2998,-0.1571,-3.0408,1,-0.0305,1.2280,43.4508,0.5910,-0.0428,-1,0.0442,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,173692,1245050481,8.9721,-1.6158,-2.7176,1,-0.0294,8.7183,13.5266,1.8709,2.7911,1,-0.0689,0.4202
9996,173692,1244979327,39.4543,-1.1310,-2.0568,1,0.0417,0.0000,47.7209,-0.1834,1.7965,-1,-0.0794,0.0000
9997,173692,1245305163,40.6034,-1.5051,-1.6758,-1,0.0635,0.2012,44.4382,-1.0010,1.5380,1,-0.0671,1.1499
9998,173692,1245041468,37.2853,1.0024,-2.1486,1,0.0361,0.0000,41.0945,-0.1200,0.8850,-1,-0.0291,0.0000


In [47]:
df=particles

fig_5 = px.histogram(
    particles,
    x='pt2',
    nbins=10000,
    title='Histograma momento transversal 2',
    labels={'pt2': 'Momento transversal pt <sub>2<sub> [GeV/c]', 'count': 'Frecuencia'},
    opacity=0.75,
    color_discrete_sequence=['#636EFA']
)

fig_5.update_layout(
    xaxis_title='pt <sub>2</sub> [GeV/c]',
    yaxis_title='Frecuencia',
    bargap=0.05,
    font=dict(family='Arial', size=14),
    title_font=dict(size=18, family='Arial', color='darkblue')
)

fig_5.show()

In [48]:
fig_5 = px.histogram(
    particles,
    x='pt1',
    nbins=10000,
    title='Histograma momento transversal 1',
    labels={'pt1': 'Momento transversal pt1 [GeV/c]', 'count': 'Frecuencia'},
    opacity=0.75,
    color_discrete_sequence=['#636EFA']
)

fig_5.update_layout(
    xaxis_title='pt<sub>1</sub> [GeV/c]',
    yaxis_title='Frecuencia',
    bargap=0.05,
    font=dict(family='Arial', size=14),
    title_font=dict(size=18, family='Arial', color='darkblue')
)

fig_5.show()

In [49]:
# Cuenta cada valor de Q1 y Q2 por separado
q1 = particles["Q1"].value_counts().sort_index() # Series: índice = carga, valor = frecuencia
q2 = particles["Q2"].value_counts().sort_index()

#  Organiza en una tabla ordenada (-1 primero, luego +1)
charges = [-1, 1] # orden explícito
table = pd.DataFrame({
    "Charge": charges,
    "Q1_count": [q1.get(c, 0) for c in charges],
    "Q2_count": [q2.get(c, 0) for c in charges],
})

# Muestra la tabla
print(table.to_string(index=False))

 Charge  Q1_count  Q2_count
     -1      5156      4808
      1      4844      5192


In [50]:
particles["Q1"] = particles["Q1"].astype(int)
particles["Q2"] = particles["Q2"].astype(int)

charges = [-1, 1]
q1_df = particles[particles["Q1"].isin(charges)]
q2_df = particles[particles["Q2"].isin(charges)]

# Histograma para Q1
fig_q1 = px.histogram(
    q1_df,
    x="Q1",
    category_orders={"Q1": charges},
    labels={"Q1": "Carga", "count": "Frecuencia"},
    title="Histograma de Q1",
    color_discrete_sequence=["#636EFA"],
    opacity=0.8
)
fig_q1.update_layout(xaxis_title="Carga", yaxis_title="Frecuencia", width=500, height=500,
    bargap=0.2)

fig_q1.show()

# Histograma para Q2
fig_q2 = px.histogram(
    q2_df,
    x="Q2",
    category_orders={"Q2": charges},
    labels={"Q2": "Carga", "count": "Frecuencia"},
    title="Histograma de Q2",
    color_discrete_sequence=["#636EFA"],
    opacity=0.8
)
fig_q2.update_layout(xaxis_title="Carga", yaxis_title="Frecuencia", width=500, height=500,
    bargap=0.2)
fig_q2.show()


In [51]:

df['QT'] = particles['Q1'] + particles['Q2']

df[["Event","Run","QT"]]

,Event,Run,QT
0,74969122,165617,0
1,75138253,165617,0
2,75887636,165617,0
3,75779415,165617,0
4,75098104,165617,0
...,...,...,...
9995,1245050481,173692,2
9996,1244979327,173692,0
9997,1245305163,173692,0
9998,1245041468,173692,0


In [52]:
# Cuenta cada valor de Q1 y Q2 por separado
QT = df["QT"].value_counts().sort_index() # Series: índice = carga, valor = frecuencia

 #  Organiza en una tabla ordenada
charges2 = [0, 2, -2] # orden explícito
table3 = pd.DataFrame({
    "Value": charges2,
    "QT_count": [QT.get(c, 0) for c in charges2],

})

# Muestra la tabla
print(table3.to_string(index=False))

 Value  QT_count
     0      9664
     2       186
    -2       150


In [53]:
#Calculamos masa invariante por partes (para entenderle)

df["pt1*pt2"]=particles['pt1'] * particles['pt2']
df["eta1-eta2"]=particles['eta1']-particles['eta2']
df["phi1-phi2"]=particles['phi1']-particles['phi2']

df["cosh"]=np.cosh(df['eta1-eta2'])
df["cos"]=np.cos(df['phi1-phi2'])

df["M^2"]=2*df["pt1*pt2"]*(df["cosh"]-df["cos"])
df["M"]=np.sqrt(df["M^2"])
df["M_r"]=np.round(df["M"])

df=df.drop(columns=["pt1","pt2","eta1","eta2","Q1","Q2","dxy1","dxy2","phi1","phi2","iso1","iso2","QT"])
df


,Run,Event,pt1*pt2,eta1-eta2,phi1-phi2,cosh,cos,M^2,M,M_r
0,165617,74969122,1873.466435,0.5561,3.0729,1.158650,-0.997642,8079.478521,89.885919,90.0
1,165617,75138253,701.691642,-2.4374,4.8578,5.765319,0.144899,7887.602846,88.812177,89.0
2,165617,75887636,959.597528,-1.7911,-3.2156,3.081411,-0.997263,7827.769550,88.474683,88.0
3,165617,75779415,1918.578493,-0.5167,-3.2826,1.136486,-0.990075,8159.947752,90.332429,90.0
4,165617,75098104,1794.509350,-0.7481,-2.9980,1.293123,-0.989708,8193.124926,90.515882,91.0
...,...,...,...,...,...,...,...,...,...,...
9995,173692,1245050481,121.362008,-3.4867,-5.5087,16.354267,0.714781,3796.078838,61.612327,62.0
9996,173692,1244979327,1882.794705,-0.9476,-3.8533,1.483591,-0.757248,8438.078560,91.859015,92.0
9997,173692,1245305163,1804.342010,-0.5041,-3.2138,1.129772,-0.997394,7676.270391,87.614328,88.0
9998,173692,1245041468,1532.220761,1.1224,-3.0336,1.698858,-0.994174,8252.641020,90.844048,91.0


In [54]:
df["M_r"].describe()

,M_r
count,10000.000000
mean,87.973100
std,8.843427
min,60.000000
25%,87.000000
50%,90.000000
75%,92.000000
max,120.000000


In [55]:
fig = px.histogram(
    df,
    x="M_r",
    nbins=len(df["M_r"].unique()),
    title="$Histograma de Masa Invariante M_r$",
    labels={"M_r": "M_r [GeV/c^2]", "count": "Frecuencia"},
    opacity=0.75,
    color_discrete_sequence=["#636EFA"]
)

fig.update_layout(
    bargap=0.05,
    xaxis_title="$M_r [GeV/c^2]$",
    yaxis_title="Frecuencia"
)

fig.show()

In [56]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import curve_fit

#Histograma
masas = df["M_r"].values #AQUI COLOCAN SUS DATOS
bin_w = 1.0

lo_edge = np.floor(masas.min() / bin_w) * bin_w
hi_edge = np.ceil (masas.max() / bin_w) * bin_w
bins = np.arange(lo_edge, hi_edge + bin_w, bin_w)
counts, edges = np.histogram(masas, bins=bins)
centros = 0.5 * (edges[:-1] + edges[1:])

#Breit-Wigner
def breit_wigner(m, M, Gamma):
    num = M**2 * Gamma**2
    denom = (m**2 - M**2)**2 + M**2 * Gamma**2
    return num / denom
#Normalizamos
def modelo(m, M, Gamma, N, a, b):
    return N * breit_wigner(m, M, Gamma)+ a + b * m


mask = (centros > 80) & (centros < 100)
x_fit = centros[mask]
y_fit = counts[mask]
sigma = np.sqrt(np.where(y_fit > 0, y_fit, 1))

p0 = [91.2, 2.5, y_fit.max()*5, y_fit.min(), 0.0]

popt, pcov = curve_fit(modelo, x_fit, y_fit, p0=p0, sigma=sigma, absolute_sigma=True, maxfev=10000)

M_Z, Gamma_Z, N_sig, a_bg, b_bg = popt
dM_Z, dGamma_Z = np.sqrt(np.diag(pcov))[:2]

print(f"M_Z = {M_Z:.3f} ± {dM_Z:.3f} GeV") #IMPRIME EL PICO
print(f"Gamma_Z = {Gamma_Z:.3f} ± {dGamma_Z:.3f} GeV") #IMPRIME EL ANCHO

#Creamos figura
m_plot = np.linspace(lo_edge, hi_edge, 2000)
total_y = modelo(m_plot, *popt)
signal_y = N_sig * breit_wigner(m_plot, M_Z, Gamma_Z)

hist = go.Histogram(
    x=masas,
    xbins=dict(start=lo_edge, end=hi_edge, size=bin_w),
    name="Datos",
    opacity=0.75,
    marker=dict(color="rgba(0,0,200,0.65)")
)

total_fit = go.Scatter(
    x=m_plot, y=total_y,
    mode="lines", name="Ajuste total",
    line=dict(color="orange")
)


signal_Z = go.Scatter(
    x=m_plot, y=signal_y,
    mode="lines", name="Señal Z",
    line=dict(color="green", dash="dash")
)

fig = go.Figure(data=[hist, total_fit, signal_Z])
fig.update_layout(
    title="Masa invariante: histograma completo y ajuste del bosón Z",
    xaxis_title="Masa invariante [GeV/c²]",
    yaxis_title=f"Eventos por bin (Δm = {bin_w} GeV)",
    bargap=0.05,
    template="plotly_white"
)
fig.show()

M_Z = 91.386 ± 0.032 GeV
Gamma_Z = 4.182 ± 0.107 GeV


In [57]:
df["p_x1"]=particles['pt1']*np.cos(particles['phi1'])
df["p_x2"]=particles['pt2']*np.cos(particles['phi2'])


df["p_y1"]=particles['pt1']*np.sin(particles['phi1'])
df["p_y2"]=particles['pt2']*np.sin(particles['phi2'])


df["p_z1"]=particles['pt1']*np.sinh(particles['eta1'])
df["p_z2"]=particles['pt2']*np.sinh(particles['eta2'])

df["p_x"]=df['p_x1']+df['p_x2']
df["p_y"]=df['p_y1']+df['p_y2']
df["p_z"]=df['p_z1']+df['p_z2']

m_mu = 0.105658 # GeV
df["E_1"]=np.sqrt((particles['pt1']*np.cosh(particles['eta1']))**2+(m_mu)**2)
df["E_2"]=np.sqrt((particles['pt2']*np.cosh(particles['eta2']))**2+(m_mu)**2)
df["E"]=df["E_1"]+df["E_2"]

df["m^2"]=df["E"]**2-df["p_x"]**2-df["p_y"]**2-df["p_z"]**2
df["m"]=np.sqrt(df["m^2"])
df["m_r"]=np.round(df["m"])


df[['Event','Run','p_x1','p_x2','p_y1','p_y2','p_z1','p_z2','p_x','p_y','p_z','E_1','E_2','E','m^2','m','m_r']]

,Event,Run,p_x1,p_x2,p_y1,p_y2,p_z1,p_z2,p_x,p_y,p_z,E_1,E_2,E,m^2,m,m_r
0,74969122,165617,-46.133428,30.075362,29.400655,-16.379515,-24.398698,-39.641341,-16.058066,13.021141,-64.040039,59.899911,52.385714,112.285625,8079.523376,89.886169,90.0
1,75138253,165617,-23.663390,-11.647553,6.676405,-26.053854,-94.126653,11.267068,-35.310943,-19.377449,-82.859585,97.284985,30.682679,127.967664,7887.664090,88.812522,89.0
2,75887636,165617,7.497983,-9.295325,-30.840217,28.770052,-150.344100,-14.685350,-1.797341,-2.070165,-165.029450,153.657731,33.612343,187.270074,7827.845354,88.475111,88.0
3,75779415,165617,37.817182,-47.572449,-12.209859,8.229456,-30.761477,-9.503704,-9.755267,-3.980403,-40.265181,50.254348,49.205624,99.459972,8159.992412,90.332676,90.0
4,75098104,165617,-41.090192,43.411009,-4.155672,-1.859127,-6.514920,27.200637,2.320817,-6.014798,20.685717,41.810631,51.262636,93.073266,8193.170046,90.516131,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1245050481,173692,-8.177653,-12.704232,-3.691147,4.644501,-21.681888,42.880360,-20.881885,0.953353,21.198472,23.465166,44.963378,68.428544,3796.128383,61.612729,62.0
9996,1244979327,173692,-18.428948,-10.679567,-34.885752,46.510549,-54.763423,-8.801159,-29.108515,11.624797,-63.564582,67.495818,48.525827,116.021645,8438.124441,91.859264,92.0
9997,1245305163,173692,-4.255676,1.457148,-40.379764,44.414303,-86.944243,-52.292424,-2.798527,4.034540,-139.236666,95.958056,68.624065,164.582122,7676.316312,87.614590,88.0
9998,1245041468,173692,-20.364689,26.024715,-31.232564,31.803649,43.955938,-4.943184,5.660026,0.571086,39.012754,57.639650,41.390870,99.030520,8252.686910,90.844300,91.0


In [58]:
fig = px.histogram(
    df,
    x="m_r",
    nbins=len(df["m_r"].unique()),
    title="$Histograma de Masa Invariante (m_r)$",
    labels={"m_r": "m_r [GeV/c^2]", "count": "Frecuencia"},
    opacity=0.75,
    color_discrete_sequence=["#636EFA"]
)

fig.update_layout(
    bargap=0.05,
    xaxis_title="$m_r [GeV/c^2]$",
    yaxis_title="Frecuencia"
)

fig.show()

In [59]:
df_fil = particles[particles["Q1"] + particles["Q2"] == 0].copy()

df_fil


,Run,Event,pt1,eta1,phi1,Q1,dxy1,iso1,pt2,eta2,...,iso2,QT,pt1*pt2,eta1-eta2,phi1-phi2,cosh,cos,M^2,M,M_r
0,165617,74969122,54.7055,-0.4324,2.5742,1,-0.0745,0.4999,34.2464,-0.9885,...,3.4221,0,1873.466435,0.5561,3.0729,1.158650,-0.997642,8079.478521,89.885919,90.0
1,165617,75138253,24.5872,-2.0522,2.8666,-1,-0.0554,0.0000,28.5389,0.3852,...,0.0000,0,701.691642,-2.4374,4.8578,5.765319,0.144899,7887.602846,88.812177,89.0
2,165617,75887636,31.7386,-2.2595,-1.3323,-1,0.0879,0.0000,30.2344,-0.4684,...,0.0000,0,959.597528,-1.7911,-3.2156,3.081411,-0.997263,7827.769550,88.474683,88.0
3,165617,75779415,39.7394,-0.7123,-0.3123,1,0.0585,0.0000,48.2790,-0.1956,...,0.0000,0,1918.578493,-0.5167,-3.2826,1.136486,-0.990075,8159.947752,90.332429,90.0
4,165617,75098104,41.2998,-0.1571,-3.0408,1,-0.0305,1.2280,43.4508,0.5910,...,0.0000,0,1794.509350,-0.7481,-2.9980,1.293123,-0.989708,8193.124926,90.515882,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,173692,1244153861,38.1252,-0.4428,-2.2370,-1,0.0311,0.0000,40.7930,0.6067,...,0.3160,0,1555.241284,-1.0495,-3.2382,1.603168,-0.995337,8082.604019,89.903304,90.0
9996,173692,1244979327,39.4543,-1.1310,-2.0568,1,0.0417,0.0000,47.7209,-0.1834,...,0.0000,0,1882.794705,-0.9476,-3.8533,1.483591,-0.757248,8438.078560,91.859015,92.0
9997,173692,1245305163,40.6034,-1.5051,-1.6758,-1,0.0635,0.2012,44.4382,-1.0010,...,1.1499,0,1804.342010,-0.5041,-3.2138,1.129772,-0.997394,7676.270391,87.614328,88.0
9998,173692,1245041468,37.2853,1.0024,-2.1486,1,0.0361,0.0000,41.0945,-0.1200,...,0.0000,0,1532.220761,1.1224,-3.0336,1.698858,-0.994174,8252.641020,90.844048,91.0


In [60]:
df_fil["pf_x1"]=df_fil['pt1']*np.cos(df_fil['phi1'])
df_fil["pf_x2"]=df_fil['pt2']*np.cos(df_fil['phi2'])


df_fil["pf_y1"]=df_fil['pt1']*np.sin(df_fil['phi1'])
df_fil["pf_y2"]=df_fil['pt2']*np.sin(df_fil['phi2'])


df_fil["pf_z1"]=df_fil['pt1']*np.sinh(df_fil['eta1'])
df_fil["pf_z2"]=df_fil['pt2']*np.sinh(df_fil['eta2'])

df_fil["pf_x"]=df_fil['pf_x1']+df_fil['pf_x2']
df_fil["pf_y"]=df_fil['pf_y1']+df_fil['pf_y2']
df_fil["pf_z"]=df_fil['pf_z1']+df_fil['pf_z2']

#df_fil["pt_f"]=np.hypot(df_fil['pf_x'],df_fil['pf_y'])
#df_fil['pt']=np.sqrt((df_fil['pf_x'])**2+(df_fil['pf_y'])**2)
df_fil['pt_f']=df_fil['pt1']/df_fil['pt2']

m_mu = 0.105658 # GeV
df_fil["Ef_1"]=np.sqrt((df_fil['pt1']*np.cosh(df_fil['eta1']))**2+(m_mu)**2)
df_fil["Ef_2"]=np.sqrt((df_fil['pt2']*np.cosh(df_fil['eta2']))**2+(m_mu)**2)
df_fil["Ef"]=df_fil["Ef_1"]+df_fil["Ef_2"]

df_fil["mf^2"]=df_fil["Ef"]**2-df_fil["pf_x"]**2-df_fil["pf_y"]**2-df_fil["pf_z"]**2
df_fil["mf"]=np.sqrt(df_fil["mf^2"])
df_fil["mf_r"]=np.round(df_fil["mf"])

df_fil['phi1'] = np.arctan2(df_fil["pf_y1"], df_fil["pf_x1"])
df_fil['phi2'] = np.arctan2(df_fil["pf_y2"], df_fil["pf_x2"])
df_fil['phi'] = np.abs(np.mod(df_fil["phi1"] - df_fil["phi2"] + np.pi, 2*np.pi) - np.pi)
#df_fil['delta_phi']=np.arccos(np.cos(df_fil['phi1']-df_fil['phi2']))

df_fil[['Event','Run','pf_x1','pf_x2','pf_y1','pf_y2','pf_z1','pf_z2','pf_x','pf_y','pf_z','Ef_1','Ef_2','Ef','mf^2','mf','mf_r','phi','pt_f']]

,Event,Run,pf_x1,pf_x2,pf_y1,pf_y2,pf_z1,pf_z2,pf_x,pf_y,pf_z,Ef_1,Ef_2,Ef,mf^2,mf,mf_r,phi,pt_f
0,74969122,165617,-46.133428,30.075362,29.400655,-16.379515,-24.398698,-39.641341,-16.058066,13.021141,-64.040039,59.899911,52.385714,112.285625,8079.523376,89.886169,90.0,3.072900,1.597409
1,75138253,165617,-23.663390,-11.647553,6.676405,-26.053854,-94.126653,11.267068,-35.310943,-19.377449,-82.859585,97.284985,30.682679,127.967664,7887.664090,88.812522,89.0,1.425385,0.861533
2,75887636,165617,7.497983,-9.295325,-30.840217,28.770052,-150.344100,-14.685350,-1.797341,-2.070165,-165.029450,153.657731,33.612343,187.270074,7827.845354,88.475111,88.0,3.067585,1.049751
3,75779415,165617,37.817182,-47.572449,-12.209859,8.229456,-30.761477,-9.503704,-9.755267,-3.980403,-40.265181,50.254348,49.205624,99.459972,8159.992412,90.332676,90.0,3.000585,0.823120
4,75098104,165617,-41.090192,43.411009,-4.155672,-1.859127,-6.514920,27.200637,2.320817,-6.014798,20.685717,41.810631,51.262636,93.073266,8193.170046,90.516131,91.0,2.998000,0.950496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,1244153861,173692,-23.561598,21.999345,-29.973021,34.352550,-17.438948,26.295598,-1.562253,4.379529,8.856650,41.924443,48.533890,90.458333,8082.648913,89.903553,90.0,3.044985,0.934602
9996,1244979327,173692,-18.428948,-10.679567,-34.885752,46.510549,-54.763423,-8.801159,-29.108515,11.624797,-63.564582,67.495818,48.525827,116.021645,8438.124441,91.859264,92.0,2.429885,0.826772
9997,1245305163,173692,-4.255676,1.457148,-40.379764,44.414303,-86.944243,-52.292424,-2.798527,4.034540,-139.236666,95.958056,68.624065,164.582122,7676.316312,87.614590,88.0,3.069385,0.913705
9998,1245041468,173692,-20.364689,26.024715,-31.232564,31.803649,43.955938,-4.943184,5.660026,0.571086,39.012754,57.639650,41.390870,99.030520,8252.686910,90.844300,91.0,3.033600,0.907306


In [61]:
import plotly.express as px

fig = px.histogram(
    df_fil,
    x=df_fil['phi'],
    nbins=1000,
    range_x=[0, 3.5],
    title="$Distribución del balance Δφ (antes de aplicar cortes)$"
)

fig.update_layout(width=700, height=700,
    xaxis_title=r"$Δφ$",
    yaxis_title="Eventos / bin",

)

fig.show()

In [79]:
ang=2.8

ang_b=df_fil['phi']>ang
candidatosang = ang_b.sum()
print(f'Eventos que pasan: {candidatosang}')

Eventos que pasan: 6857


In [63]:
import plotly.express as px

fig = px.histogram(
    df_fil,
    x=df_fil['pt_f'],
    nbins=2000,
    range_x=[0, 10],
    title="$Razón de P_{t1}/p_{t2}$ (antes de aplicar cortes)"
)

fig.update_layout(width=700, height=700,
    xaxis_title=r"$p_T\;\mathrm{[GeV]}$",
    yaxis_title="Eventos",

)

fig.show()

In [64]:
thr=10

sel_b= (0.5<df_fil['pt_f']) & (df_fil['pt_f']<1)
candidatospt = sel_b.sum()
print(f'Eventos que pasan: {candidatospt}')

Eventos que pasan: 4270


In [77]:
df_long = pd.concat([
    df_fil[["iso1"]].rename(columns={"iso1": "iso"}).assign(variable="iso1"),
    df_fil[["iso2"]].rename(columns={"iso2": "iso"}).assign(variable="iso2")
])


fig = px.histogram(
    df_long,
    x="iso",
    color="variable",
    nbins=1000,
    range_x=[0, 5],
    barmode="overlay",
    opacity=0.6,
    title="$Distribución combinada de iso_1 e iso_2$",
    color_discrete_map={
        "iso1": "blue",
        "iso2": "red"
    }
)

fig.update_layout(
    width=700,
    height=700,
    xaxis_title="iso",
    yaxis_title="Eventos / bin",
    template="plotly_white"
)

fig.show()

In [85]:
sel_c= (df_fil['iso1']<1) & (df_fil['iso2']<1)
candidatosiso = sel_b.sum()
print(f'Eventos que pasan: {candidatosiso}')

Eventos que pasan: 4270


In [96]:
met = np.hypot(df_fil["pf_x"], df_fil["pf_y"])

mask = (0<df_fil['pt_f']) & (df_fil['pt_f']<2) \
     & (60 < df_fil["mf"]) & (df_fil["mf"] < 120) \
     & (df_fil['iso1']<0.5)\
     & (df_fil['iso2']<0.5)\
     & (df_fil['phi'] > ang) \
     & (met < 20)
print("Eventos que pasan todos los cortes:", mask.sum())

Eventos que pasan todos los cortes: 3964


In [97]:
masas = df_fil["mf"][mask]
bin_w = 0.5 # GeV
edges = np.arange(70, 110+bin_w, bin_w)
centers = 0.5*(edges[1:] + edges[:-1])
counts, _ = np.histogram(masas, bins=edges)
N_tot = counts.sum()


def signal_bin(lo, hi, m0, gamma, Nsig):
    # área de Cauchy en cada bin
    return Nsig * (cauchy.cdf(hi, m0, gamma/2) - cauchy.cdf(lo, m0, gamma/2))

def model(lo, hi, m0, gamma, Nsig, a0, a1):
    signal = signal_bin(lo, hi, m0, gamma, Nsig)
    bkg = (a0 + a1*0.5*(lo+hi)) * (hi - lo)
    return signal + bkg


def model_centers(x, m0, gamma, Nsig, a0, a1):
    idx = np.searchsorted(edges, x) - 1
    lo, hi = edges[idx], edges[idx+1]
    return model(lo, hi, m0, gamma, Nsig, a0, a1)


p0 = [91.2, 2.5, N_tot*0.9, 1.0, 0.0]
bounds = ([88, 1.5, 0, 0, -1.0],
          [94, 4.0, N_tot, 10, 1.0])

pars, cov = curve_fit(model_centers, centers, counts, p0=p0, bounds=bounds)
perr = np.sqrt(np.diag(cov))
m0, gamma, Nsig, a0, a1 = pars

print(f"M_Z = {m0:.3f} ± {perr[0]:.3f} GeV")
print(f"Γ_Z = {gamma:.3f} ± {perr[1]:.3f} GeV")
print(f"N_Z^obs = {Nsig:.0f} ± {perr[2]:.0f} (≤ 3 720, coherente)")
print(f"Fondo = a0={a0:.2f} a1={a1:.3f} eventos·GeV⁻¹")


x_plot = np.linspace(70, 110, 800)
sig_pdf = Nsig * cauchy.pdf(x_plot, m0, gamma/2)
bkg_pdf = a0 + a1 * x_plot
tot_pdf = sig_pdf + bkg_pdf
sig_curve = sig_pdf * bin_w
tot_curve = tot_pdf * bin_w


fig = go.Figure()
fig.add_trace(go.Histogram(
    x=masas,
    xbins=dict(start=70, end=110, size=bin_w),
    marker=dict(color="rgba(0,0,200,0.65)"),
    opacity=0.75,
    name="Datos"
))
fig.add_trace(go.Scatter(x=x_plot, y=tot_curve,
    mode="lines", line=dict(width=2), name="Ajuste total"))
fig.add_trace(go.Scatter(x=x_plot, y=sig_curve,
    mode="lines", line=dict(width=2, dash="dash"), name="Señal BW"))
fig.update_layout(
    template="plotly_white",
    bargap=0.02,
    title="Pico Z → ℓ⁺ℓ⁻ : histograma y ajuste con fondo lineal",
    xaxis_title="Masa invariante m [GeV]",
    yaxis_title=f"Eventos / {bin_w} GeV"
)
fig.show()

M_Z = 90.865 ± 0.033 GeV
Γ_Z = 3.833 ± 0.108 GeV
N_Z^obs = 3849 ± 89 (≤ 3 720, coherente)
Fondo = a0=10.00 a1=-0.054 eventos·GeV⁻¹
